In [1]:
import logging
from pathlib import Path

import pandas as pd
from dotenv import dotenv_values

from generator_drainage_units import run_generator_drainage_units

logging.basicConfig(level=logging.INFO)
pd.set_option("future.no_silent_downcasting", True)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Define case and base_dir
config = dotenv_values("..\\.env")
base_dir = Path(config["BASE_DIR"])
# case_name
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
# case_name = "Leuvenumse_beek"
case_name = "Leuvenumse_beek_dinf"
# case_name = "West"
# case_name = "Barneveldsebeek"
# case_name = "Barneveldsebeek2"

case_path = Path(base_dir, case_name)
dir_basisdata = "0_basisdata"
dir_results = "1_resultaat"
ghg_file_name = "GHG_2000-2010_L1.NC"

In [ ]:
resolution = 10.0

gdu = run_generator_drainage_units(
    path=case_path,
    dir_basisdata=dir_basisdata,
    dir_results=dir_results,
    method="pyflwdir",
    ghg_file_name=ghg_file_name,
    read_results=True,
    write_results=True,
    preprocess=True,
    process=True,
    postprocess=False,
    resolution=resolution,
    buffer_waterways=1.5*resolution,
    depth_waterways=0.2,
    smooth_distance=25.0,
    iterations=20000,
    iteration_group=100,
)

In [ ]:
gdu = run_generator_drainage_units(
    path=case_path,
    dir_basisdata=dir_basisdata,
    dir_results=dir_results,
    method="pyflwdir",
    ghg_file_name=ghg_file_name,
    read_results=True,
    write_results=True,
    preprocess=False,
    process=False,
    postprocess=True,
    resolution=resolution,
    buffer_waterways=1.5*resolution,
    depth_waterways=0.2,
    smooth_distance=25.0,
    iterations=20000,
    iteration_group=100,
)

In [ ]:
gdu.generate_folium_map(
    order_labels=True,
    zmin=7.5,
    zmax=25.0,
    dx=0.0,
    dy=-13.0,
)